In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
# Model
m = gp.Model("district")
m

Restricted license - for non-production use only - expires 2025-11-24


<gurobi.Model Continuous instance district: 0 constrs, 0 vars, No parameter changes>

In [3]:
state = 'WA'
population = "districting_2020\\2020\\{}\\counties\\graph\\{}.population".format(state, state)
dimacs = "districting_2020\\2020\\{}\\counties\\graph\\{}.dimacs".format(state, state)

with open('Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts = int(parts[1])


with open(population, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)

num_districts


FileNotFoundError: [Errno 2] No such file or directory: 'districting_2020\\2020\\WA\\counties\\graph\\WA.population'

In [4]:
## Reading distances from file
filename = "districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)
d = np.genfromtxt(filename, delimiter=',', skip_header=1)
d = d[:, 1:]

## Reading populations from file
with open(population, 'r') as file:
    # Read the total population from the first line
    total_population_line = next(file).strip()  # Removes newline character at the end
    total_population = int(total_population_line.split('=')[1].strip())
    
    # Initialize a dictionary to store the rest of the data
    p = {}
    
    # Iterate over the remaining lines
    for line in file:
        # Each line is expected to have two values, separated by a space
        key, value = line.strip().split()  # Split by whitespace
        p[int(key)] = int(value)  # Convert both to integers and store in the dictionary

## Reading neighbors from file
neighbors = set()
with open(dimacs, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('e'):
            _, node1, node2 = line.split()
            node1 = int(node1)
            node2 = int(node2)
            if (node1, node2) not in neighbors and (node2, node1) not in neighbors:
                neighbors.add((node1, node2))

list(neighbors)

FileNotFoundError: districting_2020\2020\WA\counties\graph\WA_distances.csv not found.

In [5]:
adjacency_matrix_file = r"districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)

# Set decision variable x for each node i whether it is in district j
x = {}
for i in range(num_counties):
    for j in range(num_districts):
        x[i, j] = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

## Set decision variable y for each edge in neighbors, between nodes i and k, for each district j
y = {}
for i, k in neighbors:
    for j in range(num_districts):
        y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# for i in range(num_counties):
#     for k in range(num_counties):
#         for j in range(num_districts):
#             y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# Set slack variable
slack = {}
for j in range(num_districts):
    slack[j] = m.addVar(obj=1, name="slack")




alpha = 1  # Weight of the balanced weight term
y


NameError: name 'num_counties' is not defined

In [6]:
# Set objective function
m.setObjective(gp.quicksum(y[i, k, j] * d[i, k] for i, k in neighbors for j in range(num_districts)) \
             + alpha * (gp.quicksum(slack[j] for j in range(num_districts))), gp.GRB.MINIMIZE)


NameError: name 'neighbors' is not defined

In [7]:
# Set objective function
m.setObjective(gp.quicksum(y[i, k, j] * d[i, k] for i, k in neighbors for j in range(num_districts)) \
             + alpha * (gp.quicksum(slack[j] for j in range(num_districts))), gp.GRB.MINIMIZE)


NameError: name 'neighbors' is not defined

In [9]:
# Write LP formulation to a file
m.write("districts.lp")

# Read the LP formulation from the file
with open("districts.lp", "r") as f:
    lp_formulation = f.read()

# Print LP formulation
print("LP Formulation:")
print(lp_formulation)

# Solve
m.optimize()

LP Formulation:
\ Model district
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
 
Subject To
Bounds
End

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [10]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nObjective Value: %g' % m.ObjVal)
        for i in range(num_counties):
            for j in range(num_districts):
                print("x_{}_{} = {}".format(i, j, x[i, j].X))
        for j in range(num_districts):
            print("slack_{} = {}".format(j, slack[j].X))
    else:
        print('No solution')

printSolution()


Objective Value: 0


NameError: name 'num_counties' is not defined

In [12]:
import geopandas as gpd
import matplotlib.pyplot as plt

path_to_visualize = "districting_2020/2020/{}/counties/maps/{}_counties.shp".format(state, state)
# Load the shapefile
counties_gdf = gpd.read_file(path_to_visualize)


counties_gdf.plot(column='NAME20', legend = True)
plt.show()


DriverError: districting_2020/2020/WA/counties/maps/WA_counties.shp: No such file or directory

In [13]:
def get_district_counties():
    # Initialize a dictionary to hold counties for each district
    district_counties = {j: set() for j in range(num_districts)}

    # Iterate over x variables to find counties assigned to each district
    for i in range(num_counties):
        for j in range(num_districts):
            if x[i, j].X > 0.5:  # County is part of the district
                district_counties[j].add(i)
    
    return district_counties

# Assuming the model has been optimized
if m.status == GRB.OPTIMAL:
    district_counties = get_district_counties()
    for j, counties in district_counties.items():
        print(f"District {j} contains counties: {counties}")
else:
    print("No solution")


NameError: name 'num_counties' is not defined

In [ ]:

import networkx as nx

path_to_visualize = "districting_2020/2020/{}/counties/maps/{}_counties.shp".format(state, state)
# Load the shapefile
counties_gdf = gpd.read_file(path_to_visualize)

# Example: Assuming 'county_id' is a column in your GeoDataFrame that matches keys used in your model
counties_gdf['county_id'] = range(len(counties_gdf))
counties_gdf['district'] = -1  # Initialize with -1 or another placeholder value

# Assuming district_counties is already filled from your optimization model
for district, counties in district_counties.items():
    for county in counties:
        # Update the 'district' column based on your county-to-district assignments
        # You may need to adjust the condition to match your GeoDataFrame's county identifiers
        counties_gdf.loc[counties_gdf['county_id'] == county, 'district'] = district


import matplotlib.pyplot as plt

# Ensure that the 'district' column exists and has been correctly populated
fig, ax = plt.subplots(figsize=(10, 10))
counties_gdf.plot(column='district', ax=ax, legend=True, categorical=True, legend_kwds={'title': 'District'})
plt.show()
